# Collect Demonstration from Keyboard

Collect demonstration data for the given environment.
The task is to pick a mug and place it on the plate. The environment recognizes the success if the mug is on the plate, gthe ripper opened, and the end-effector positioned above the mug.

<img src="./media/teleop_v2.gif" width="480" height="360">

Use WASD for the xy plane, RF for the z-axis, QE for tilt, and ARROWs for the rest of rthe otations. 

SPACEBAR will change your gripper's state, and Z key will reset your environment with discarding the current episode data.

For overlayed images, 
- Top Right: Agent View 
- Bottom Right: Egocentric View
- Top Left: Left Side View
- Bottom Left: Top View

In [4]:
# Cell 1 - 设置环境变量（必须第一个运行）
import os

# 1. 设置DISPLAY
os.environ['DISPLAY'] = ':0'
os.environ['XAUTHORITY'] = os.path.expanduser('~/.Xauthority')
print(f"✓ DISPLAY设置为: {os.environ['DISPLAY']}")

# 2. 强制使用GPU渲染（关键！）
os.environ['MUJOCO_GL'] = 'egl'  # EGL后端GPU加速
print(f"✓ MUJOCO_GL: egl (GPU硬件加速)")

# 3. NVIDIA GPU优化
os.environ['__GL_SYNC_TO_VBLANK'] = '0'  # 关闭垂直同步
os.environ['__GL_YIELD'] = 'NOTHING'      # 减少CPU等待
print("✓ NVIDIA GPU优化已启用")

# 4. OpenGL性能优化
os.environ['__GL_FSAA_MODE'] = '0'        # 关闭抗锯齿
os.environ['__GL_LOG_MAX_ANISO'] = '0'    # 关闭各向异性过滤
print("✓ OpenGL性能优化已启用")

✓ DISPLAY设置为: :0
✓ MUJOCO_GL: egl (GPU硬件加速)
✓ NVIDIA GPU优化已启用
✓ OpenGL性能优化已启用


In [5]:
import sys
import random
import numpy as np
import os
from PIL import Image
from mujoco_env.y_env2 import SimpleEnv2
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

/home/wzy/envs/lerobot-mujoco/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# If you want to randomize the object positions, set this to None
# If you fix the seed, the object positions will be the same every time
SEED = 0 
# SEED = None <- Uncomment this line to randomize the object positions

REPO_NAME = 'omy_pnp_language'
NUM_DEMO = 20 # Number of demonstrations to collect
ROOT = "./demo_data_language" # The root directory to save the demonstrations

In [7]:
xml_path = './asset/example_scene_y2.xml'
# Define the environment
PnPEnv = SimpleEnv2(xml_path, seed = SEED, state_type = 'joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[31] n_qvel:[28] n_qacc:[28] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[23]
 [0/23] [world] mass:[0.00]kg
 [1/23] [front_object_table] mass:[1.00]kg
 [2/23] [camera] mass:[0.00]kg
 [3/23] [camera2] mass:[0.00]kg
 [4/23] [camera3] mass:[0.00]kg
 [5/23] [link1] mass:[2.06]kg
 [6/23] [link2] mass:[3.68]kg
 [7/23] [link3] mass:[2.39]kg
 [8/23] [link4] mass:[1.40]kg
 [9/23] [link5] mass:[1.40]kg
 [10/23] [link6] mass:[0.65]kg
 [11/23] [camera_center] mass:[0.00]kg
 [12/23] [tcp_link] mass:[0.32]kg
 [13/23] [rh_p12_rn_r1] mass:[0.07]kg
 [14/23] [rh_p12_rn_r2] mass:[0.02]kg
 [15/23] [rh_p12_rn_l1] mass:[0.07]kg
 [16/23] [rh_p12_rn_l2] mass:[0.02]kg
 [17/23] [body_obj_mug_5] mass:[0.00]kg
 [18/23] [object_mug_5] mass:[0.08]kg
 [19/23] [body_obj_plate_11] mass:[0.00]kg
 [20/23] [object_plate_11] mass:[0.10]kg
 [21/23] [body_obj_mug_6] mass:[0.00]kg
 [22/23] [object_mug

## Define Dataset Fatures and Create your dataset!
The dataset is contained as follows:
```
fps = 20,
features={
    "observation.image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channels"],
    },
    "observation.wrist_image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channel"],
    },
    "observation.state": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["state"], # x, y, z, roll, pitch, yaw
    },
    "action": {
        "dtype": "float32",
        "shape": (7,),
        "names": ["action"], # 6 joint angles and 1 gripper
    },
    "obj_init": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["obj_init"], # just the initial position of the object. Not used in training.
    },
},
```


This will make the dataset on './demo_data' folder, which will look like this,

```
.
├── data
│   ├── chunk-000
│   │   ├── episode_000000.parquet
│   │   └── ...
├── meta
│   ├── episodes.jsonl
│   ├── info.json
│   ├── stats.json
│   └── tasks.jsonl
└── 
```


In [8]:
create_new = True
if os.path.exists(ROOT):
    print(f"Directory {ROOT} already exists.")
    ans = input("Do you want to delete it? (y/n) ")
    if ans == 'y':
        import shutil
        shutil.rmtree(ROOT)
    else:
        create_new = False


if create_new:
    dataset = LeRobotDataset.create(
                repo_id=REPO_NAME,
                root = ROOT, 
                robot_type="omy",
                fps=20, # 20 frames per second
                features={
                    "observation.image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channels"],
                    },
                    "observation.wrist_image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channel"],
                    },
                    "observation.state": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["state"], # x, y, z, roll, pitch, yaw
                    },
                    "action": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["action"], # 6 joint angles and 1 gripper
                    },
                    "obj_init": {
                        "dtype": "float32",
                        "shape": (9,),
                        "names": ["obj_init"], # just the initial position of the object. Not used in training.
                    },
                },
                image_writer_threads=10,
                image_writer_processes=5,
        )
else:
    print("Load from previous dataset")
    dataset = LeRobotDataset(REPO_NAME, root=ROOT)

Process Process-5:
Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wzy/envs/lerobot-mujoco/lib/python3.10/site-packages/lerobot/common/datasets/image_writer.py", line 103, in worker_process
    t.join()
  File "/home/wzy/envs/lerobot-mujoco/lib/python3.10/site-packages/lerobot/common/datasets/image_writer.py", line 103, in worker_process
    t.join()
  File "/usr/lib/python3.10/threading.py", line 1096, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.10/threading.py", line 1096, in join
   

## Keyboard Control
You can teleop your robot with keyboard and collect dataset
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```
Reseting your environment will remove the cache data of the current demonstration and restart collection.

### Now let's teleop our robot and collect data!

**To receive the success signal, you have to release the gripper and move upwards above the mug!**

In [9]:
action = np.zeros(7)
episode_id = 0
record_flag = False # Start recording when the robot starts moving
while PnPEnv.env.is_viewer_alive() and episode_id < NUM_DEMO:
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # check if the episode is done
        done = PnPEnv.check_success()
        if done: 
            # Save the episode data and reset the environment
            dataset.save_episode()
            PnPEnv.reset()
            episode_id += 1
        # Teleoperate the robot and get delta end-effector pose with gripper
        action, reset  = PnPEnv.teleop_robot()
        if not record_flag and sum(action) != 0:
            record_flag = True
            print("Start recording")
        if reset:
            # Reset the environment and clear the episode buffer
            # This can be done by pressing 'z' key
            # PnPEnv.reset(seed=SEED)
            PnPEnv.reset()
            dataset.clear_episode_buffer()
            record_flag = False
        # Step the environment
        # Get the end-effector pose and images
        agent_image,wrist_image = PnPEnv.grab_image()
        # # resize to 256x256
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)
        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))
        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)
        joint_q = PnPEnv.step(action)
        action = PnPEnv.q[:7] # 6 joint angles and 1 gripper
        action = action.astype(np.float32)
        if record_flag:
            # Add the frame to the dataset
            dataset.add_frame( {
                    "observation.image": agent_image,
                    "observation.wrist_image": wrist_image,
                    "observation.state": joint_q[:6], 
                    "action": action,
                    "obj_init": PnPEnv.obj_init_pose,
                    # "task": PnPEnv.instruction,
                }, task = PnPEnv.instruction
            )
        PnPEnv.render(teleop=True, idx=episode_id)

Start recording


KeyboardInterrupt: 

In [ ]:
PnPEnv.env.close_viewer()

In [ ]:
# Clean up the images folder
import shutil
shutil.rmtree(dataset.root / 'images')

FileNotFoundError: [Errno 2] No such file or directory: 'demo_data_language/images'